# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse Shipments               Customer  \
0  Jun 12 2023  9:54AM  264542        15  30017146  Alliance Pharma, Inc.   
1  Jun 12 2023  9:54AM  264542        15  30017147  Alliance Pharma, Inc.   
2  Jun 12 2023  9:54AM  264542        15  30017148  Alliance Pharma, Inc.   
3  Jun 12 2023  9:54AM  264542        15  30017149  Alliance Pharma, Inc.   
4  Jun 12 2023  9:54AM  264542        15  30017151  Alliance Pharma, Inc.   
5  Jun 12 2023  9:54AM  264542        15  30017150  Alliance Pharma, Inc.   
6  Jun 12 2023  9:54AM  264542        15  30017170  Alliance Pharma, Inc.   
7  Jun 12 2023  9:54AM  264542        15  30017171  Alliance Pharma, Inc.   
8  Jun 12 2023  9:54AM  264542        15  30017172  Alliance Pharma, Inc.   
9  Jun 12 2023  9:54AM  264542        15  30017164  Alliance Pharma, Inc.   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
15  264539      Executing          1
16  264539       Released         16
17  264540       Released         12
18  264541       Released          1
19  264542       Released         27

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Executing  Released
id                                 
264538                NaN       1.0
264539                1.0      16.0
264540                NaN      12.0
264541                NaN       1.0
264542                NaN      27.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Executing  Released
id                                 
264516                1.0       0.0
264517                1.0       0.0
264523               10.0       9.0
264527                0.0       1.0
264528                0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus  Executing  Released
id                                 
264516                  1         0
264517                  1         0
264523                 10         9
264527                  0         1
264528                  0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Executing  Released
0               264516          1         0
1               264517          1         0
2               264523         10         9
3               264527          0         1
4               264528          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus      id Executing Released
0               264516         1         
1               264517         1         
2               264523        10        9
3               264527                  1
4               264528                  1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                         Customer
0    Jun 12 2023  9:54AM  264542        15            Alliance Pharma, Inc.
27   Jun 12 2023  9:51AM  264541        19              AdvaGen Pharma, Ltd
28   Jun 12 2023  9:49AM  264540        19              AdvaGen Pharma, Ltd
40   Jun 12 2023  9:43AM  264539        15  Brookfield Pharmaceuticals, LLC
57   Jun 12 2023  9:42AM  264538        20              AdvaGen Pharma, Ltd
58   Jun 12 2023  9:33AM  264537        10                        Hush Hush
59   Jun 12 2023  9:31AM  264536        10                        Hush Hush
62   Jun 12 2023  9:30AM  264535        18                          Seqirus
63   Jun 12 2023  9:29AM  264534        10                        Hush Hush
106  Jun 12 2023  9:19AM  264533        19     ThermoFisher Scientific Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                         Customer  \
0  Jun 12 2023  9:54AM  264542        15            Alliance Pharma, Inc.   
1  Jun 12 2023  9:51AM  264541        19              AdvaGen Pharma, Ltd   
2  Jun 12 2023  9:49AM  264540        19              AdvaGen Pharma, Ltd   
3  Jun 12 2023  9:43AM  264539        15  Brookfield Pharmaceuticals, LLC   
4  Jun 12 2023  9:42AM  264538        20              AdvaGen Pharma, Ltd   
5  Jun 12 2023  9:33AM  264537        10                        Hush Hush   
6  Jun 12 2023  9:31AM  264536        10                        Hush Hush   
7  Jun 12 2023  9:30AM  264535        18                          Seqirus   
8  Jun 12 2023  9:29AM  264534        10                        Hush Hush   
9  Jun 12 2023  9:19AM  264533        19     ThermoFisher Scientific Inc.   

  Executing Released  
0                 27  
1                  1  
2                 12  
3         1       16  
4                  1  
5                  1  
6                  3  
7                  1  
8                 43  
9         1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                         Customer  \
0  Jun 12 2023  9:54AM  264542        15            Alliance Pharma, Inc.   
1  Jun 12 2023  9:51AM  264541        19              AdvaGen Pharma, Ltd   
2  Jun 12 2023  9:49AM  264540        19              AdvaGen Pharma, Ltd   
3  Jun 12 2023  9:43AM  264539        15  Brookfield Pharmaceuticals, LLC   
4  Jun 12 2023  9:42AM  264538        20              AdvaGen Pharma, Ltd   
5  Jun 12 2023  9:33AM  264537        10                        Hush Hush   
6  Jun 12 2023  9:31AM  264536        10                        Hush Hush   
7  Jun 12 2023  9:30AM  264535        18                          Seqirus   
8  Jun 12 2023  9:29AM  264534        10                        Hush Hush   
9  Jun 12 2023  9:19AM  264533        19     ThermoFisher Scientific Inc.   

  Released Executing  
0       27            
1        1            
2       12            
3       16         1  
4        1            
5        1            
6        3            
7        1            
8       43            
9                  1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                         Customer  \
0  Jun 12 2023  9:54AM  264542        15            Alliance Pharma, Inc.   
1  Jun 12 2023  9:51AM  264541        19              AdvaGen Pharma, Ltd   
2  Jun 12 2023  9:49AM  264540        19              AdvaGen Pharma, Ltd   
3  Jun 12 2023  9:43AM  264539        15  Brookfield Pharmaceuticals, LLC   
4  Jun 12 2023  9:42AM  264538        20              AdvaGen Pharma, Ltd   

  Released Executing  
0       27            
1        1            
2       12            
3       16         1  
4        1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


Date      id Warehouse                         Customer  \
0  Jun 12 2023  9:54AM  264542        15            Alliance Pharma, Inc.   
1  Jun 12 2023  9:51AM  264541        19              AdvaGen Pharma, Ltd   
2  Jun 12 2023  9:49AM  264540        19              AdvaGen Pharma, Ltd   
3  Jun 12 2023  9:43AM  264539        15  Brookfield Pharmaceuticals, LLC   
4  Jun 12 2023  9:42AM  264538        20              AdvaGen Pharma, Ltd   

   Released  Executing  
0      27.0        NaN  
1       1.0        NaN  
2      12.0        NaN  
3      16.0        1.0  
4       1.0        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                         Customer  \
0  Jun 12 2023  9:54AM  264542        15            Alliance Pharma, Inc.   
1  Jun 12 2023  9:51AM  264541        19              AdvaGen Pharma, Ltd   
2  Jun 12 2023  9:49AM  264540        19              AdvaGen Pharma, Ltd   
3  Jun 12 2023  9:43AM  264539        15  Brookfield Pharmaceuticals, LLC   
4  Jun 12 2023  9:42AM  264538        20              AdvaGen Pharma, Ltd   

   Released  Executing  
0      27.0        0.0  
1       1.0        0.0  
2      12.0        0.0  
3      16.0        1.0  
4       1.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing
Warehouse                              
10         1587188      77.0       10.0
12          264531       0.0        1.0
15          529081      43.0        1.0
18          793591       3.0        0.0
19         1058131      13.0        2.0
20          264538       1.0        0.0
22          264516       0.0        1.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing
0        10  1587188      77.0       10.0
1        12   264531       0.0        1.0
2        15   529081      43.0        1.0
3        18   793591       3.0        0.0
4        19  1058131      13.0        2.0
5        20   264538       1.0        0.0
6        22   264516       0.0        1.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing
0        10      77.0       10.0
1        12       0.0        1.0
2        15      43.0        1.0
3        18       3.0        0.0
4        19      13.0        2.0
5        20       1.0        0.0
6        22       0.0        1.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   77.0
1        12  Released    0.0
2        15  Released   43.0
3        18  Released    3.0
4        19  Released   13.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12    15   18    19   20   22
Status                                         
Executing  10.0  1.0   1.0  0.0   2.0  0.0  1.0
Released   77.0  0.0  43.0  3.0  13.0  1.0  0.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12    15   18    19   20   22
0          Executing  10.0  1.0   1.0  0.0   2.0  0.0  1.0
1           Released  77.0  0.0  43.0  3.0  13.0  1.0  0.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12    15   18    19   20   22
0  Executing  10.0  1.0   1.0  0.0   2.0  0.0  1.0
1   Released  77.0  0.0  43.0  3.0  13.0  1.0  0.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()